## Imports

In [ ]:
# pip install selenium webdriver-manager

# o objetivo deste notebook é olhar para os links que foram scrape e tentar extrair as peças do procedimento

# agora vou primeiro passar isto tudo para um repositório no git lol

import pandas as pd
import polars as pl
import numpy as np

import shutil
from pathlib import Path

import os

from copy import deepcopy

from utilities.funcoes_scrape import download_pecas_flat, get_covid_legal_frameworks
from utilities.schema import contracts_schema, procedimentos_schema
# from utilities.funcoes import preprocessar_contratos

In [2]:
# already preprocessed

contratos = pl.read_parquet(
    "../../data/contratos_cleaned_prepro.parquet",
    columns=[
        "N.º Procedimento (ID BASE)", "N.º Contrato", "Tipo(s) de contrato_LIMPO", "Objeto_LIMPO", "Objeto_LIMPO_2", "Objeto"
    ]
    )


procedimentos = pl.read_csv("../../data/impic_data/procedimentos.csv",
                            separator=";",
                            schema_overrides=procedimentos_schema(),
                            columns=[
                                "ContractingProcedureAliasID", "Número do Anúncio (Nº/Ano)", "Descrição/designação do procedimento", "Data de Criação", 
                                "URL Peças do Procedimento", "Nº de Lotes", "Regime de Contratação"
                                     ],
                            null_values=["NULL"]) \
                            .with_columns(
                                pl.col("Data de Criação").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F")
                                        ) \
                            .unique(subset="ContractingProcedureAliasID")



contratos_raw = pl.read_csv("../../data/impic_data/contratos.csv", separator=";", schema_overrides=contracts_schema(),
    columns=['N.º Procedimento (ID BASE)', 'N.º Contrato', 'Data da decisão adjudicação', 
             'Data Celebração', "Contratação Excluída", "Medidas Especiais", "Tipo de procedimento",
              "Ao abrigo dos critérios materiais", 'Preço BASE (€)', 'Preço Contratual (€)'], 
    null_values=["NULL"]) \
    .with_columns(
        pl.col("Data da decisão adjudicação").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
        pl.col('Data Celebração').replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
    ) \
    .with_columns(
        pl.col("Data Celebração").dt.year().alias("contract_year")
    ) \
    .join(procedimentos, how="left", left_on="N.º Procedimento (ID BASE)", right_on="ContractingProcedureAliasID", coalesce=True) \
    .filter(pl.col("Contratação Excluída") == False) \
    .filter(pl.col("Tipo de procedimento").is_in(["Concurso público", "Concurso limitado por prévia qualificação"])) \
    .filter(pl.col("Medidas Especiais").is_null()) \
    .filter(~pl.col("Regime de Contratação").str.contains_any(get_covid_legal_frameworks())) \
    .unique(subset=['N.º Procedimento (ID BASE)', 'N.º Contrato'])#.select(['N.º Procedimento (ID BASE)', 'N.º Contrato', "contract_year"])


In [42]:
contratos = contratos.join(contratos_raw, how="inner", on=['N.º Procedimento (ID BASE)', 'N.º Contrato'], coalesce=True) \
                        .with_columns(
                                pl.when(pl.col("URL Peças do Procedimento").is_null()).then(pl.lit(1)).otherwise(pl.lit(0)).alias("no_url"),
                                pl.when(pl.col("URL Peças do Procedimento").str.contains("vortal")).then(pl.lit(1)).otherwise(pl.lit(0)).alias("vortal"),
                                pl.when(pl.col("URL Peças do Procedimento").str.contains("donwload")).then(pl.lit(1)).otherwise(pl.lit(0)).alias("download")
                        )

In [43]:
contratos.filter(pl.col("download")==1).shape

(14136, 39)

In [44]:
contratos.filter(pl.col("vortal")==1).shape

(11686, 39)

In [90]:
# ver os que já foram extraídos

ids_extraidos = []
for item in os.listdir('../../docs_pecas'):
    ids_extraidos.append(item)

In [91]:
len(ids_extraidos)

4410

In [92]:
# ver os que podemos extrair com download as pecas do procedimento

contratos_pecas_download = contratos.filter(
    (pl.col('download')==1) &
    (~pl.col("N.º Procedimento (ID BASE)").cast(pl.Utf8).is_in(ids_extraidos))
).unique(subset='N.º Procedimento (ID BASE)')

In [93]:
contratos_pecas_download_pd = contratos_pecas_download.to_pandas()

In [94]:
contratos_pecas_download_pd.shape

(4004, 39)

In [88]:
to_extract = contratos_pecas_download_pd[:1000]

In [89]:
to_extract.apply(download_pecas_flat, axis=1, out_root="../../docs_pecas")

0      {'ok': True, 'N.º Procedimento (ID BASE)': '52...
1      {'ok': True, 'N.º Procedimento (ID BASE)': '50...
2      {'ok': True, 'N.º Procedimento (ID BASE)': '51...
3      {'ok': True, 'N.º Procedimento (ID BASE)': '53...
4      {'ok': True, 'N.º Procedimento (ID BASE)': '44...
                             ...                        
995    {'ok': True, 'N.º Procedimento (ID BASE)': '46...
996    {'ok': True, 'N.º Procedimento (ID BASE)': '44...
997    {'ok': True, 'N.º Procedimento (ID BASE)': '53...
998    {'ok': True, 'N.º Procedimento (ID BASE)': '45...
999    {'ok': True, 'N.º Procedimento (ID BASE)': '44...
Length: 1000, dtype: object

#### Caso vortal

In [ ]:
contratos_extract_vortal= contratos_extract[contratos_extract['URL Peças do Procedimento'].str.contains('vortal', case=False, na=False, regex=True)]
contratos_pecas_vortal = contratos_extract_vortal.drop_duplicates(subset='N.º Procedimento (ID BASE)')